## Create 50 x 90 grid
Geographical area is divided up into 0.01 degree squares to create an overlay grid (fishnet). Using 50 x 90 (as opposed to the exact 43 x 90) to align this new grid to the original model of the Shenzhen taxi data.

In [4]:
import arcpy, os, math
arcpy.env.overwriteOutput = True

# workspace
root = r"C:\Temp\shenzhen_work"
gdb  = os.path.join(root, "shenzhen_work.gdb")
if not os.path.exists(root):
    os.makedirs(root)
if not arcpy.Exists(gdb):
    arcpy.management.CreateFileGDB(root, os.path.basename(gdb))
arcpy.env.workspace = gdb

# Force lon/lat degrees
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4326)  # GCS WGS 84

# Rectangle and grid size
xmin, ymin, xmax, ymax = 113.75, 22.44, 114.65, 22.87
ROWS, COLS = 50, 90

fishnet = "grid_50x90_wgs84"
labels  = "grid_50x90_wgs84_lbl"
for fc in (fishnet, labels):
    if arcpy.Exists(fc):
        arcpy.management.Delete(fc)

# *** Standard orientation: origin = bottom-left; y-axis UP; corner = top-right ***
arcpy.management.CreateFishnet(
    out_feature_class=fishnet,
    origin_coord=f"{xmin} {ymin}",        # bottom-left
    y_axis_coord=f"{xmin} {ymax}",        # points upward
    cell_width="", cell_height="",
    number_rows=str(ROWS), number_columns=str(COLS),
    corner_coord=f"{xmax} {ymax}",        # top-right
    labels="NO_LABELS", template="", geometry_type="POLYGON"
)

# Centroids
arcpy.management.FeatureToPoint(fishnet, labels, "INSIDE")

# Verify extent
ext = arcpy.Describe(fishnet).extent
print("Extent:", round(ext.XMin,5), round(ext.YMin,5), round(ext.XMax,5), round(ext.YMax,5))
# Expect: 113.75 22.44 114.65 22.87

# Add row/col with TOP-LEFT origin (0,0)
for f in ("row", "col", "cell_id"):
    if f not in [fld.name for fld in arcpy.ListFields(fishnet)]:
        arcpy.management.AddField(fishnet, f, "LONG")

dx = (xmax - xmin) / COLS
dy = (ymax - ymin) / ROWS

with arcpy.da.UpdateCursor(fishnet, ["SHAPE@XY", "row", "col", "cell_id"]) as cur:
    for (cx, cy), r, c, cid in cur:
        # Compute column index: left → right
        col = int(math.floor((cx - xmin) / dx))
        col = min(max(col, 0), COLS - 1)

        # Compute row index: top → bottom (flip)
        row_from_bottom = int(math.floor((cy - ymin) / dy))
        row = (ROWS - 1) - row_from_bottom  # invert row order so row 0 is top

        # Clamp and compute 0-indexed ID
        row = min(max(row, 0), ROWS - 1)
        cell_id = row * COLS + col

        cur.updateRow([(cx, cy), row, col, cell_id])

print("Done: (0,0) = top-left, (39,49) = bottom-right, 0-indexed numbering.")


Extent: 113.75 22.44 114.65 22.87
Done: (0,0) = top-left, (39,49) = bottom-right, 0-indexed numbering.


# Project grid and district areas to meters

In [5]:
utm49 = arcpy.SpatialReference(32649) # 32649 = WGS 84 datum + UTM projection + Zone 49 + Northern Hemisphere, tells ArcGIS how to project data
arcpy.management.Project("grid_50x90_wgs84", "grid_50x90_utm49", utm49) # project grid to meters
arcpy.management.Project("shenzhen_districts_wgs84", "districts_utm49", utm49) # project Shenzhen district polygons to meters
arcpy.management.MultipartToSinglepart("districts_utm49", "districts_utm49_sp") # separate multipart components of districts

<Result 'C:\\Temp\\shenzhen_work\\shenzhen_work.gdb\\districts_utm49_sp'>

# Tabulate intersection (ti) of grid and districts

## Create area-by-district table

In [9]:
# Ensure each cell in table has a unique cell_id
arcpy.management.AddField("grid_50x90_utm49", "cell_id", "LONG")
arcpy.management.CalculateField("grid_50x90_utm49", "cell_id", "!row!*90 + !col!", "PYTHON3")

# Calculate overlap between grid cells and Shenzhen districts
arcpy.analysis.TabulateIntersection(
    in_zone_features="grid_50x90_utm49", # 50 x 90 grid
    zone_fields="cell_id", # unique cell identifier
    in_class_features="districts_utm49_sp", # Shenzhen district boundary polygons
    out_table="ti_grid_district", # output table will have one row per (cell,district) with AREA in m^2
    class_fields="name"  # district name column from districts_utm49
)


<Result 'C:\\Temp\\shenzhen_work\\shenzhen_work.gdb\\ti_grid_district'>

## Select majority district per cell based on overlap percentage

In [13]:
import arcpy
from collections import defaultdict

# --- Step 1: Build dictionary of max overlap per cell ---
best = defaultdict(lambda: (0.0, None))  # (area, district)

with arcpy.da.SearchCursor("ti_grid_district", ["cell_id", "name", "AREA"]) as cur:
    for cid, dname, area in cur:
        if area > best[cid][0]:
            best[cid] = (area, dname)

# Now `best` looks like:
# { 101: (123456.0, "Bao'an District"),
#   102: (98765.0, "Nanshan District"), ... }

# --- Step 2: Add fields to grid if missing ---
grid = "grid_50x90_utm49"

if "cell_area" not in [f.name for f in arcpy.ListFields(grid)]:
    arcpy.management.AddField(grid, "cell_area", "DOUBLE")
    arcpy.management.CalculateGeometryAttributes(grid, [["cell_area","AREA"]])

for fld in ("district","overlap_m2","overlap_pct"):
    if fld not in [f.name for f in arcpy.ListFields(grid)]:
        arcpy.management.AddField(grid, fld, "TEXT" if fld=="district" else "DOUBLE")

# --- Step 3: Update grid with majority district + overlap ---
with arcpy.da.UpdateCursor(grid, ["cell_id","district","overlap_m2","overlap_pct","cell_area"]) as cur:
    for cid, dist, a_m2, pct, carr in cur:
        if cid in best:
            amax, dname = best[cid]  # unpack (area, district)
            pctv = (amax / carr * 100.0) if carr else None
            cur.updateRow([cid, dname, amax, pctv, carr])


# Add numeric district_id

In [23]:
import arcpy

# Path to your grid feature class (the one with district field)
grid = r"C:\Temp\shenzhen_work\shenzhen_work.gdb\grid_50x90_utm49"

# --- Step 1: Get the distinct district names (ignore nulls) ---
districts = sorted({row[0] for row in arcpy.da.SearchCursor(grid, ["district"]) if row[0]})

print("Found districts:", districts)
# Example: ['Bao\'an District', 'Futian District', 'Guangming District', 'Longgang District',
#           'Longhua District', 'Luohu District', 'Nanshan District', 'Pingshan District', 'Yantian District']

# --- Step 2: Build lookup dictionary (name → numeric ID) ---
district_map = {name: i for i, name in enumerate(districts)}
print("Mapping:", district_map)
# Example: {'Bao\'an District': 0, 'Futian District': 1, ..., 'Yantian District': 8}

# --- Step 3: Add numeric field (if missing) ---
if "district_id" not in [f.name for f in arcpy.ListFields(grid)]:
    arcpy.management.AddField(grid, "district_id", "SHORT")

# --- Step 4: Update the field using the mapping ---
with arcpy.da.UpdateCursor(grid, ["district", "district_id"]) as cur:
    for dname, did in cur:
        new_id = district_map.get(dname)
        cur.updateRow([dname, new_id])

print("✅ Added 'district_id' field with numeric codes.")


Found districts: ['Bao’an District', 'Dapeng District', 'Futian District', 'Guangming District', 'Longgang District', 'Longhua District', 'Luohu District', 'Nanshan District', 'Pingshan District', 'Yantian District']
Mapping: {'Bao’an District': 0, 'Dapeng District': 1, 'Futian District': 2, 'Guangming District': 3, 'Longgang District': 4, 'Longhua District': 5, 'Luohu District': 6, 'Nanshan District': 7, 'Pingshan District': 8, 'Yantian District': 9}
✅ Added 'district_id' field with numeric codes.


# Export grid mapping

In [24]:
import csv
with open(r"C:\Users\rober\Downloads\district_id_mapping.csv", "w", newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["district", "district_id"])
    for name, did in district_map.items():
        writer.writerow([name, did])
print("✅ Wrote district_id mapping CSV.")


✅ Wrote district_id mapping CSV.


# Export grid table

In [26]:
import arcpy
import os

# Path to your feature class (grid layer with district data)
grid = r"C:\Temp\shenzhen_work\shenzhen_work.gdb\grid_50x90_utm49"
# C:\Temp\shenzhen_work\shenzhen_work.gdb
# Output CSV path (must be a regular folder, not inside a .gdb)
out_csv = r"C:\Users\rober\Downloads\grid_to_district_ArcGIS_table_raw.csv"
# out_csv = r"C:\Temp\grid_50x90_districts.csv"

# Export table
arcpy.conversion.TableToTable(
    in_rows=grid,
    out_path=os.path.dirname(out_csv),
    out_name=os.path.basename(out_csv)
)

print("✅ Export complete:", out_csv)


✅ Export complete: C:\Users\rober\Downloads\grid_to_district_ArcGIS_table_raw.csv
